<a href="https://colab.research.google.com/github/PabloCGarcia/bioDeepLearning/blob/main/Clase08_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clase de introducción de deepchem, experimentos con deep learning con datasets conocidos

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  27265      0 --:--:-- --:--:-- --:--:-- 27265


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
!pip install --pre deepchem

     |████████████████████████████████| 552kB 4.3MB/s 


In [3]:
import deepchem as dc
dc.__version__

'2.5.0.dev'

In [4]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv', splitter="scaffold")
train_dataset, valid_dataset, test_dataset = datasets

In [11]:
tasks

['measured log solubility in mols per litre']

In [6]:
transformers

In [7]:
test_dataset

<DiskDataset X.shape: (113,), y.shape: (113, 1), w.shape: (113, 1), ids: ['c1cc2ccc3cccc4ccc(c1)c2c34' 'Cc1cc(=O)[nH]c(=S)[nH]1'
 'Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 ' ...
 'c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43' 'Cc1occc1C(=O)Nc2ccccc2'
 'OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O '], task_names: ['measured log solubility in mols per litre']>

In [8]:
test_dataset.y[:5]

array([[-1.60114461],
       [ 0.20848251],
       [-0.01602738],
       [-2.82191713],
       [-0.52891635]])

In [9]:
i=0
for X, y, w, id in test_dataset.itersamples():
    print(y, id)
    i+=1
    if i == 5:
      break

[-1.60114461] c1cc2ccc3cccc4ccc(c1)c2c34
[0.20848251] Cc1cc(=O)[nH]c(=S)[nH]1
[-0.01602738] Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 
[-2.82191713] c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
[-0.52891635] C1=Cc2cccc3cccc1c23


In [10]:
test_dataset.to_dataframe().head()

,X,y,w,ids
0,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.601145,1.0,c1cc2ccc3cccc4ccc(c1)c2c34
1,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.208483,1.0,Cc1cc(=O)[nH]c(=S)[nH]1
2,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.016027,1.0,Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4
3,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-2.821917,1.0,c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
4,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.528916,1.0,C1=Cc2cccc3cccc1c23


In [ ]:
tasks

In [ ]:
transformers

In [12]:
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2)
model.fit(train_dataset, nb_epoch=100)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(324,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(324, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(966,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_16:

0.11201744079589844

In [13]:

metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(train_dataset, [metric], transformers))
print("Test set score:", model.evaluate(test_dataset, [metric], transformers))

Training set score: {'pearson_r2_score': 0.905058320851338}
Test set score: {'pearson_r2_score': 0.6648839130136818}


In [14]:
solubilities = model.predict_on_batch(test_dataset.X[:10])
for molecule, solubility in zip(test_dataset.ids, solubilities):
    print(solubility, molecule)

[-1.4874398] c1cc2ccc3cccc4ccc(c1)c2c34
[0.43210125] Cc1cc(=O)[nH]c(=S)[nH]1
[-0.71713287] Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 
[-1.954528] c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
[-1.2010237] C1=Cc2cccc3cccc1c23
[2.0218284] CC1CO1
[-0.06135403] CCN2c1ccccc1N(C)C(=S)c3cccnc23 
[-1.0053833] CC12CCC3C(CCc4cc(O)ccc34)C2CCC1=O
[-1.4865197] Cn2cc(c1ccccc1)c(=O)c(c2)c3cccc(c3)C(F)(F)F
[-0.37259924] ClC(Cl)(Cl)C(NC=O)N1C=CN(C=C1)C(NC=O)C(Cl)(Cl)Cl 


In [16]:
predicciones = model.predict_on_batch(valid_dataset.X[:10])
for molecule, solubility, solesperada in zip(valid_dataset.ids, predicciones, valid_dataset.y):
    print(molecule, solubility, solesperada)

Nc1ncnc2nc[nH]c12  [1.6309068] [0.36138149]
Nc1nc(O)nc2nc[nH]c12  [1.1414341] [-0.25844]
Fc1cccc(F)c1C(=O)NC(=O)Nc2cc(Cl)c(F)c(Cl)c2F  [-0.07383418] [-2.13532331]
Fc1cccc(F)c1C(=O)NC(=O)Nc2ccc(Cl)cc2 [-0.27588558] [-1.52566284]
CC(=O)OC3(CCC4C2C=C(C)C1=CC(=O)CCC1(C)C2CCC34C)C(C)=O  [-0.59326947] [-1.2014783]
CC(=O)C3(C)CCC4C2C=C(C)C1=CC(=O)CCC1(C)C2CCC34C [-0.5000251] [-1.1627697]
CC34CCC1C(CCC2CC(O)CCC12C)C3CCC4=O [-0.05009018] [-0.62568785]
CC12CCC(O)CC1CCC3C2CCC4(C)C3CCC4=O [-0.05009024] [-0.74278137]
CCOP(=S)(OCC)Oc1nc(Cl)n(n1)C(C)C [-0.3461938] [-0.38279138]
Nc1nc[nH]n1 [1.9786507] [1.63973305]
